In [ ]:
# # Install dependencies if needed (run once, then comment out)
!pip install ultralytics opencv-python numpy

# # Import required libraries
# try:
#     import cv2 as cv
#     from ultralytics import YOLO
#     import numpy as np
#     print("Libraries imported successfully!")
# except ImportError as e:
#     print(f"Error importing libraries: {e}")
#     print("Please ensure ultralytics, opencv-python, and numpy are installed.")
#     raise SystemExit

# # Load Haar Cascade for face detection
# face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
# if face_cascade.empty():
#     print("Error: Could not load face cascade classifier.")
#     raise SystemExit

# # Load YOLOv8 model (pre-trained, nano for speed)
# try:
#     model = YOLO('yolov8n.pt')  # Nano model for real-time performance
# except Exception as e:
#     print(f"Error loading YOLOv8 model: {e}")
#     print("Ensure yolov8n.pt is downloaded or check internet connection.")
#     raise SystemExit

# # Function to detect hands and count fingers
# def detect_hands_and_fingers(frame, yolo_results, gray):
#     """
#     Detects hands in person ROIs using skin color and counts fingers via contour defects.
#     Args:
#         frame: BGR image
#         yolo_results: YOLOv8 detection results
#         gray: Grayscale image
#     Returns:
#         hand_detected: Boolean indicating hand detection
#         finger_counts: Dict with 'Left' and 'Right' finger counts
#     """
#     hand_detected = False
#     finger_counts = {'Left': 0, 'Right': 0}
    
#     for result in yolo_results:
#         boxes = result.boxes.xyxy.cpu().numpy().astype(int)  # Bounding boxes
#         classes = result.boxes.cls.cpu().numpy().tolist()    # Class IDs
#         for box, cls in zip(boxes, classes):
#             if int(cls) == 0:  # Class 0 is 'person' in COCO
#                 x1, y1, x2, y2 = box
#                 # Approximate hand regions (left/right sides of person)
#                 person_width = x2 - x1
#                 left_hand_roi = frame[y1:y2, x1:x1 + person_width // 2]
#                 right_hand_roi = frame[y1:y2, x1 + person_width // 2:x2]
                
#                 # Process each ROI for hands using skin color
#                 for roi, label in [(left_hand_roi, 'Left'), (right_hand_roi, 'Right')]:
#                     if roi.size == 0:
#                         continue
#                     hsv = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
#                     # Skin color range (adjustable for lighting/skin tone)
#                     lower_skin = np.array([0, 20, 70], dtype=np.uint8)
#                     upper_skin = np.array([20, 255, 255], dtype=np.uint8)
#                     mask = cv.inRange(hsv, lower_skin, upper_skin)
                    
#                     # Reduce noise
#                     kernel = np.ones((5, 5), np.uint8)
#                     mask = cv.dilate(mask, kernel, iterations=2)
#                     mask = cv.erode(mask, kernel, iterations=1)
                    
#                     # Find contours
#                     contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#                     for contour in contours:
#                         area = cv.contourArea(contour)
#                         if area > 1000:  # Filter small noise
#                             # Draw rectangle in original frame
#                             rx, ry, rw, rh = cv.boundingRect(contour)
#                             if label == 'Left':
#                                 rx += x1
#                             else:
#                                 rx += x1 + person_width // 2
#                             ry += y1
#                             cv.rectangle(frame, (rx, ry), (rx + rw, ry + rh), (0, 0, 255), 2)  # Red
#                             # Count fingers via convexity defects
#                             hull = cv.convexHull(contour, returnPoints=False)
#                             finger_count = 0
#                             if len(hull) > 3:
#                                 defects = cv.convexityDefects(contour, hull)
#                                 if defects is not None:
#                                     for i in range(defects.shape[0]):
#                                         s, e, f, d = defects[i, 0]
#                                         start = tuple(contour[s][0])
#                                         end = tuple(contour[e][0])
#                                         far = tuple(contour[f][0])
#                                         # Filter defects by depth and angle
#                                         a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
#                                         b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
#                                         c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
#                                         angle = np.arccos((b**2 + c**2 - a**2) / (2*b*c)) * 57  # Cosine rule
#                                         if d > 1000 and angle < 90:  # Depth and angle filter
#                                             finger_count += 1
#                                     finger_count = min(finger_count + 1, 5)  # +1 for thumb
#                                 finger_counts[label] = finger_count  # Fixed: Use 'label' instead of 'axis'
#                                 cv.putText(frame, f'{label} Hand: {finger_count} fingers', (rx, ry - 10),
#                                            cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
#                                 hand_detected = True
    
#     return hand_detected, finger_counts

# # Function to estimate body parts (shoulders)
# def estimate_body_parts(frame, yolo_results):
#     """
#     Estimates shoulder positions from person bounding boxes.
#     Args:
#         frame: BGR image
#         yolo_results: YOLOv8 detection results
#     Returns:
#         body_detected: Boolean indicating body detection
#     """
#     body_detected = False
#     for result in yolo_results:
#         boxes = result.boxes.xyxy.cpu().numpy().astype(int)
#         classes = result.boxes.cls.cpu().numpy().tolist()
#         for box, cls in zip(boxes, classes):
#             if int(cls) == 0:  # 'person'
#                 x1, y1, x2, y2 = box
#                 width = x2 - x1
#                 shoulder_offset = width // 4
#                 left_shoulder = (x1 + shoulder_offset, y1 + 50)
#                 right_shoulder = (x2 - shoulder_offset, y1 + 50)
#                 cv.circle(frame, left_shoulder, 5, (255, 255, 0), -1)  # Yellow
#                 cv.circle(frame, right_shoulder, 5, (255, 255, 0), -1)
#                 cv.putText(frame, 'Left Shoulder', (left_shoulder[0], left_shoulder[1] - 10),
#                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
#                 cv.putText(frame, 'Right Shoulder', (right_shoulder[0], right_shoulder[1] - 10),
#                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
#                 body_detected = True
#     return body_detected

# # Open the webcam
# cap = cv.VideoCapture(0)
# if not cap.isOpened():
#     print("Error: Could not open webcam. Try index 1 or 2.")
#     raise SystemExit

# # Set resolution for performance
# cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

# # Process frames in real-time
# try:
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("Error: Could not read frame.")
#             break
        
#         # Flip frame for mirror-like view
#         frame = cv.flip(frame, 1)
        
#         # Convert to grayscale
#         gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
#         # Detect faces
#         faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#         face_detected = False
#         for (x, y, w, h) in faces:
#             cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue
#             cv.putText(frame, 'Face', (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
#             face_detected = True
        
#         # Run YOLOv8 for person detection
#         results = model(frame, verbose=False)
        
#         # Detect hands and fingers
#         hand_detected, finger_counts = detect_hands_and_fingers(frame, results, gray)
        
#         # Estimate body parts
#         body_detected = estimate_body_parts(frame, results)
        
#         # Display status
#         status = []
#         if face_detected: status.append("Face")
#         if hand_detected: status.append("Hands")
#         if body_detected: status.append("Body")
#         cv.putText(frame, f'Detected: {", ".join(status)}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
#         # Display frame
#         cv.imshow('Real-Time Detection', frame)
        
#         # Press 'q' to quit
#         if cv.waitKey(1) & 0xFF == ord('q'):
#             break

# except Exception as e:
#     print(f"Error: {e}")

# # Release resources
# cap.release()
# cv.destroyAllWindows()

In [ ]:
!pip install mediapipe==0.10.21


In [ ]:
# Install dependencies if needed (run once, then comment out)
# !pip install ultralytics opencv-python numpy cvzone mediapipe==0.10.21

# Import required libraries
try:
    import cv2 as cv
    from ultralytics import YOLO
    import numpy as np
    import tkinter as tk
    from threading import Thread
    try:
        from cvzone.HandTrackingModule import HandDetector
        use_cvzone = True
    except ImportError as e:
        print(f"cvzone or mediapipe import failed: {e}. Using YOLOv8 + skin color fallback.")
        use_cvzone = False
    print("Libraries imported successfully!")
except ImportError as e:
    print(f"Error importing core libraries: {e}")
    print("Please ensure ultralytics, opencv-python, numpy are installed.")
    raise SystemExit

# Initialize global exit flag
exit_flag = False

# Load Haar Cascade for face detection
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
if face_cascade.empty():
    print("Error: Could not load face cascade classifier.")
    raise SystemExit

# Load YOLOv8 model (nano for speed)
try:
    model = YOLO('yolov8n.pt')
except Exception as e:
    print(f"Error loading YOLOv8 model: {e}")
    print("Ensure yolov8n.pt is downloaded or check internet connection.")
    raise SystemExit

# Initialize cvzone HandDetector
if use_cvzone:
    try:
        hand_detector = HandDetector(maxHands=2, detectionCon=0.8)
    except Exception as e:
        print(f"cvzone HandDetector initialization failed: {e}. Using YOLOv8 + skin color fallback.")
        use_cvzone = False

# Function to detect hands and count fingers with cvzone
def detect_hands_cvzone(frame):
    """
    Detects hands, counts fingers, and recognizes gestures using cvzone.
    Args:
        frame: BGR image
    Returns:
        hand_detected: Boolean
        finger_counts: Dict {'Left': count, 'Right': count}
        gesture_labels: Dict {'Left': gesture, 'Right': gesture}
        frame: Annotated image
    """
    hand_detected = False
    finger_counts = {'Left': 0, 'Right': 0}
    gesture_labels = {'Left': '', 'Right': ''}
    
    hands, frame = hand_detector.findHands(frame, draw=True)
    for hand in hands:
        label = hand['type']
        fingers = hand_detector.fingersUp(hand)
        finger_count = sum(fingers)
        finger_counts[label] = finger_count
        
        # Draw text
        bbox = hand['bbox']
        cv.putText(frame, f'{label} Hand: {finger_count} fingers', (bbox[0], bbox[1] - 20),
                   cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        
        # Gestures
        if finger_count == 2 and fingers[1] == 1 and fingers[2] == 1:
            gesture_labels[label] = 'V Sign'
        elif finger_count == 0:
            gesture_labels[label] = 'Fist'
        elif finger_count == 3 and fingers[0] == 1 and fingers[2] == 1 and fingers[3] == 1:
            gesture_labels[label] = 'OK Sign'
        if gesture_labels[label]:
            cv.putText(frame, gesture_labels[label], (bbox[0], bbox[1] - 40),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        hand_detected = True
    
    return hand_detected, finger_counts, gesture_labels, frame

# Fallback: YOLOv8 + skin color
def detect_hands_yolo(frame, yolo_results, gray):
    """
    Fallback hand detection using YOLOv8 person ROIs and skin color.
    Args:
        frame: BGR image
        yolo_results: YOLOv8 results
        gray: Grayscale image
    Returns:
        hand_detected: Boolean
        finger_counts: Dict {'Left': count, 'Right': count}
        gesture_labels: Dict {'Left': '', 'Right': ''}
        frame: Annotated image
    """
    hand_detected = False
    finger_counts = {'Left': 0, 'Right': 0}
    gesture_labels = {'Left': '', 'Right': ''}
    
    for result in yolo_results:
        boxes = result.boxes.xyxy.cpu().numpy().astype(int)
        classes = result.boxes.cls.cpu().numpy().tolist()
        for box, cls in zip(boxes, classes):
            if int(cls) == 0:
                x1, y1, x2, y2 = box
                person_width = x2 - x1
                left_hand_roi = frame[y1:y2, x1:x1 + person_width // 2]
                right_hand_roi = frame[y1:y2, x1 + person_width // 2:x2]
                
                for roi, label in [(left_hand_roi, 'Left'), (right_hand_roi, 'Right')]:
                    if roi.size == 0:
                        continue
                    hsv = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
                    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
                    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
                    mask = cv.inRange(hsv, lower_skin, upper_skin)
                    
                    # Noise reduction
                    kernel = np.ones((5, 5), np.uint8)
                    mask = cv.dilate(mask, kernel, iterations=2)
                    mask = cv.erode(mask, kernel, iterations=1)
                    
                    # Contours
                    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
                    for contour in contours:
                        area = cv.contourArea(contour)
                        if area > 1000:
                            rx, ry, rw, rh = cv.boundingRect(contour)
                            if label == 'Left':
                                rx += x1
                            else:
                                rx += x1 + person_width // 2
                            ry += y1
                            cv.rectangle(frame, (rx, ry), (rx + rw, ry + rh), (0, 0, 255), 2)
                            hull = cv.convexHull(contour, returnPoints=False)
                            finger_count = 0
                            if len(hull) > 3:
                                defects = cv.convexityDefects(contour, hull)
                                if defects is not None:
                                    for i in range(defects.shape[0]):
                                        s, e, f, d = defects[i, 0]
                                        start = tuple(contour[s][0])
                                        end = tuple(contour[e][0])
                                        far = tuple(contour[f][0])
                                        a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                                        b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                                        c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                                        angle = np.arccos((b**2 + c**2 - a**2) / (2*b*c)) * 57
                                        if d > 1000 and angle < 90:
                                            finger_count += 1
                                    finger_count = min(finger_count + 1, 5)
                                finger_counts[label] = finger_count
                                cv.putText(frame, f'{label} Hand: {finger_count} fingers', (rx, ry - 10),
                                           cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                                hand_detected = True
    
    return hand_detected, finger_counts, gesture_labels, frame

# Body parts estimation
def estimate_body_parts(frame, yolo_results):
    """
    Estimates shoulder positions from YOLOv8 person boxes.
    Args:
        frame: BGR image
        yolo_results: YOLOv8 results
    Returns:
        body_detected: Boolean
        frame: Annotated image
    """
    body_detected = False
    for result in yolo_results:
        boxes = result.boxes.xyxy.cpu().numpy().astype(int)
        classes = result.boxes.cls.cpu().numpy().tolist()
        for box, cls in zip(boxes, classes):
            if int(cls) == 0:
                x1, y1, x2, y2 = box
                width = x2 - x1
                shoulder_offset = width // 4
                left_shoulder = (x1 + shoulder_offset, y1 + 50)
                right_shoulder = (x2 - shoulder_offset, y1 + 50)
                cv.circle(frame, left_shoulder, 5, (255, 255, 0), -1)
                cv.circle(frame, right_shoulder, 5, (255, 255, 0), -1)
                cv.putText(frame, 'Left Shoulder', (left_shoulder[0], left_shoulder[1] - 10),
                           cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                cv.putText(frame, 'Right Shoulder', (right_shoulder[0], right_shoulder[1] - 10),
                           cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                body_detected = True
    return body_detected, frame

# Tkinter exit button
def create_exit_button():
    """
    Creates Tkinter window with exit button.
    """
    root = tk.Tk()
    root.title("Control Panel")
    root.geometry("200x100")
    def exit_program():
        global exit_flag
        exit_flag = True
        root.destroy()
    button = tk.Button(root, text="Exit", command=exit_program, font=("Arial", 14), bg="red", fg="white")
    button.pack(pady=20)
    root.mainloop()

# Main loop
def main():
    """
    Main loop for real-time detection.
    """
    global exit_flag
    Thread(target=create_exit_button, daemon=True).start()
    
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam. Try index 1 or 2.")
        return
    
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    
    try:
        while not exit_flag:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame.")
                break
            
            frame = cv.flip(frame, 1)
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            
            # Faces
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            face_detected = False
            for (x, y, w, h) in faces:
                cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv.putText(frame, 'Face', (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
                face_detected = True
            
            # YOLOv8
            results = model(frame, verbose=False)
            
            # Hands
            if use_cvzone:
                hand_detected, finger_counts, gesture_labels, frame = detect_hands_cvzone(frame)
            else:
                hand_detected, finger_counts, gesture_labels, frame = detect_hands_yolo(frame, results, gray)
            
            # Body
            body_detected, frame = estimate_body_parts(frame, results)
            
            # Status
            status = []
            if face_detected: status.append("Face")
            if hand_detected: status.append("Hands")
            if body_detected: status.append("Body")
            cv.putText(frame, f'Detected: {", ".join(status)}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            
            cv.imshow('Real-Time Detection', frame)
            
            if cv.waitKey(1) & 0xFF == ord('q'):
                exit_flag = True
                break
    
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        cap.release()
        cv.destroyAllWindows()

if __name__ == "__main__":
    main()

Libraries imported successfully!
